# FOPDT-modellering – Interaktiv Colab-notatbok

Last opp en datafil (`pid_test.txt`), juster parametre med sliders,
og last ned ferdig figur som PNG.

In [ ]:
!pip install ipywidgets
from google.colab import output
output.enable_custom_widget_manager()

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, HBox, VBox, Output
from IPython.display import display, Markdown
import io
from google.colab import files

# -----------------------------
# Last opp måledatafil
# -----------------------------
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Les inn data med Pandas, håndterer desimal med komma
df = pd.read_csv(filename, sep=None, engine='python', decimal=',')
tid_data = df.iloc[:,0].values
niva_data = df.iloc[:,1].values

# -----------------------------
# Automatisk estimering fra måledata
# -----------------------------
y0_est = niva_data[0]
A_est = niva_data[-1] - y0_est

thresh = y0_est + 0.05*A_est
L_est = tid_data[np.where(niva_data > thresh)[0][0]]

thresh63 = y0_est + 0.63*A_est
T_est = tid_data[np.where(niva_data > thresh63)[0][0]]

display(Markdown(f"**Autoestimat:** A={A_est:.2f}, T={T_est:.2f}, L={L_est:.2f}, y0={y0_est:.2f}"))

# -----------------------------
# Funksjon for plotting og PNG-lagring
# -----------------------------
def plot_fopdt_save(A, T, L, y0, lagre=False):
    y_model = np.where(tid_data < L, y0, y0 + A * (1 - np.exp(-(tid_data - L)/T)))

    fig, ax = plt.subplots(figsize=(10,5))
    ax.plot(tid_data, niva_data, 'b.', markersize=3, label='Måledata')
    ax.plot(tid_data, y_model, 'r-', label=f'Modell: A={A:.2f}, T={T:.2f}, L={L:.2f}, y0={y0:.2f}')
    ax.set_xlabel('Tid [t]')
    ax.set_ylabel('Nivå / respons')
    ax.grid(True)
    ax.legend()
    plt.show()

    # PID-formler (elevene fyller selv)
    display(Markdown(
f"""
**Estimerte verdier fra modellen:**
- A = {A:.2f}
- T = {T:.2f}
- L = {L:.2f}
- y0 = {y0:.2f}
"""
    ))

    if lagre:
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        files.download('FOPDT_plot.png')

# -----------------------------
# Sliders
# -----------------------------
A_slider = FloatSlider(value=A_est, min=0, max=2*A_est, step=0.1, description='A')
T_slider = FloatSlider(value=T_est, min=1, max=2*T_est, step=0.1, description='T')
L_slider = FloatSlider(value=L_est, min=0, max=2*L_est, step=0.1, description='L')
y0_slider = FloatSlider(value=y0_est, min=0, max=2*y0_est, step=0.1, description='y0')

# -----------------------------
# Knapp for å laste ned PNG
# -----------------------------
save_button = Button(description="Last ned PNG")
def on_save_clicked(b):
    plot_fopdt_save(A_slider.value, T_slider.value, L_slider.value, y0_slider.value, lagre=True)
save_button.on_click(on_save_clicked)

# -----------------------------
# Output-widget for graf
# -----------------------------
out = Output()

# Sliders + knapp i en vertikal kolonne
controls = VBox([A_slider, T_slider, L_slider, y0_slider, save_button])

# Sett graf og controls side om side
ui = HBox([out, controls])
display(ui)

# -----------------------------
# Oppdatering av graf
# -----------------------------
def update_plot(change):
    with out:
        out.clear_output(wait=True)
        plot_fopdt_save(A_slider.value, T_slider.value, L_slider.value, y0_slider.value)

A_slider.observe(update_plot, names='value')
T_slider.observe(update_plot, names='value')
L_slider.observe(update_plot, names='value')
y0_slider.observe(update_plot, names='value')

# Første plot
update_plot(None)

# -----------------------------
# Instruksjoner til elevene
# -----------------------------
display(Markdown(
r"""
**Instruksjoner til elevene:**

1. Dra sliderne for A, T, L og y0 for å justere modellkurven slik at den passer best mulig til de blå måledataene.
2. Observer hvordan den røde kurven (modellen) endres når du justerer verdiene.
3. Når du er fornøyd med tilpasningen, kan du trykke på **Last ned PNG** for å lagre plottet.
4. Noter de estimerte verdiene av A, T, L og y0 som modellen beregner.
5. Bruk de estimerte verdiene til å fylle ut PID-formlene nedenfor:
  <ul>
  <li>$K = \dfrac{A}{\Delta u}$</li><br>
  <li>$K_p = \dfrac{T}{K(\lambda + L)}$</li><br>
  <li>$T_i = T$</li><br>
  <li>$T_d = \dfrac{L \cdot T}{L + \lambda}$</li>
</ul>
6. Diskuter i grupper hvordan valg av λ påvirker PID-parametrene og systemets respons.
"""
))

Saving pid_test.txt to pid_test (23).txt


**Autoestimat:** A=77.20, T=97.00, L=11.00, y0=5.30


**Instruksjoner til elevene:**

1. Dra sliderne for A, T, L og y0 for å justere modellkurven slik at den passer best mulig til de blå måledataene.
2. Observer hvordan den røde kurven (modellen) endres når du justerer verdiene.
3. Når du er fornøyd med tilpasningen, kan du trykke på **Last ned PNG** for å lagre plottet.
4. Noter de estimerte verdiene av A, T, L og y0 som modellen beregner.
5. Bruk de estimerte verdiene til å fylle ut PID-formlene nedenfor:
  <ul>
  <li>$K = \dfrac{A}{\Delta u}$</li><br>
  <li>$K_p = \dfrac{T}{K(\lambda + L)}$</li><br>
  <li>$T_i = T$</li><br>
  <li>$T_d = \dfrac{L \cdot T}{L + \lambda}$</li>
</ul>
6. Diskuter i grupper hvordan valg av λ påvirker PID-parametrene og systemets respons.
